# Projeto 1 - Ciência dos Dados

Nome: Pedro Henrique

Nome: Vitor Katakura

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [392]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [393]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/home/vitorhpk/www/python/cdados-p1


Carregando a base de dados com os tweets classificados manualmente:

In [394]:
stop_words = pd.read_csv('stopwords.csv', header=None)[0].to_list()
stop_words

['de',
 'a',
 'o',
 'que',
 'e',
 'do',
 'da',
 'em',
 'um',
 'para',
 'é',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'foi',
 'ao',
 'ele',
 'das',
 'tem',
 'à',
 'seu',
 'sua',
 'ou',
 'ser',
 'quando',
 'muito',
 'há',
 'nos',
 'já',
 'está',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'era',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'ter',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'estão',
 'você',
 'tinha',
 'foram',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'têm',
 'numa',
 'pelos',
 'elas',
 'havia',
 'seja',
 'qual',
 'será',
 'nós',
 'tenho',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'fosse',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo

In [395]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Target,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,"Bem, eu nem sei por onde começar. A história é...",2,NaN,NaN,NaN
1,"Não gostei!, acabou com a expectativa que os o...",2,NaN,LEGENDA,NaN
2,"Venho por meio deste, expressar meu desapontam...",1,NaN,1,Problemas com a Amazon
3,"O começo do livro é bastante interessante, qun...",2,NaN,2,Comentarios sobre o livro
4,Veder essa porcaria como se fosse História é u...,2,NaN,3,Outros


In [396]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Target,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,O livro fala de um contexto bastante interessa...,2,NaN,NaN,NaN
1,Estava muito empolgada com 90% do livro. As co...,2,NaN,LEGENDA,NaN
2,"História maluca e que não te leva a nada, uma ...",2,NaN,1,Problemas com a Amazon
3,"O autor tenta mostrar outro lado do livro, bus...",2,NaN,2,Comentarios sobre o livro
4,"Maçante, fantasioso. Uma ficção vista pelo ret...",2,NaN,3,Outros


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [397]:
legenda = {'Problemas com a Amazon': 1, 'Comentários sobre o livro': 2, 'outros': 3}

In [398]:
train = train.loc[:,['Mensagem', 'Target']]
test = test.loc[:,['Mensagem', 'Target']]
train

,Mensagem,Target
0,"Bem, eu nem sei por onde começar. A história é...",2
1,"Não gostei!, acabou com a expectativa que os o...",2
2,"Venho por meio deste, expressar meu desapontam...",1
3,"O começo do livro é bastante interessante, qun...",2
4,Veder essa porcaria como se fosse História é u...,2
...,...,...
295,Achei que o livro deu mais ênfase ao sexo do q...,2
296,Não recomendo. Honestamente não recomendo. O c...,2
297,depois de fazer o pedido que vi que era pocket...,1
298,A personagem principal parece sofrer de uma es...,2


In [399]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    texto_sem_stop_words = ''
    for palavra in text.split():
        if palavra not in stop_words:
            texto_sem_stop_words += palavra + ' '
             
    punctuation = r'[\`\"\!\-\.\:\?\;\$\'\,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    text_subbed = re.sub(emoji_pattern, '', texto_sem_stop_words)
    text_subbed = re.sub(pattern, '', text_subbed)
    return text_subbed.lower()

In [400]:
train['Mensagem'] = train['Mensagem'].apply(cleanup)
test['Mensagem'] = test['Mensagem'].apply(cleanup)

In [401]:
train

,Mensagem,Target
0,bem eu nem sei por onde começar a história é a...,2
1,não gostei acabou com a expectativa que os out...,2
2,venho por meio deste expressar meu desapontame...,1
3,o começo do livro é bastante interessante qund...,2
4,veder essa porcaria como se fosse história é u...,2
...,...,...
295,achei que o livro deu mais ênfase ao sexo do q...,2
296,não recomendo honestamente não recomendo o con...,2
297,depois de fazer o pedido que vi que era pocket...,1
298,a personagem principal parece sofrer de uma es...,2


In [402]:
test

,Mensagem,Target
0,o livro fala de um contexto bastante interessa...,2
1,estava muito empolgada com 90% do livro as coi...,2
2,história maluca e que não te leva a nada uma v...,2
3,o autor tenta mostrar outro lado do livro busc...,2
4,maçante fantasioso uma ficção vista pelo retro...,2
...,...,...
195,no início achei interessante a garota virgem ...,2
196,cheio de teorias da conspiração,2
197,por ser versão pocket se espera uma qualidade ...,3
198,o livro é uma ficcção destópica que visa criti...,2


In [403]:
train['Mensagem'] = train['Mensagem'].astype('category')
train['Target'] = train['Target'].astype('category')

train['Target'] = train['Target'].cat.rename_categories(['Problemas com a Amazon', 'Comentários sobre o livro', 'Outros'])
train.Target

0      Comentários sobre o livro
1      Comentários sobre o livro
2         Problemas com a Amazon
3      Comentários sobre o livro
4      Comentários sobre o livro
                 ...            
295    Comentários sobre o livro
296    Comentários sobre o livro
297       Problemas com a Amazon
298    Comentários sobre o livro
299    Comentários sobre o livro
Name: Target, Length: 300, dtype: category
Categories (3, object): ['Problemas com a Amazon', 'Comentários sobre o livro', 'Outros']

In [404]:
train.head(10)

,Mensagem,Target
0,bem eu nem sei por onde começar a história é a...,Comentários sobre o livro
1,não gostei acabou com a expectativa que os out...,Comentários sobre o livro
2,venho por meio deste expressar meu desapontame...,Problemas com a Amazon
3,o começo do livro é bastante interessante qund...,Comentários sobre o livro
4,veder essa porcaria como se fosse história é u...,Comentários sobre o livro
5,mais uma coletânea de artigos de jornal escrit...,Comentários sobre o livro
6,poxa comprei o livro em janeiro agora eles nao...,Problemas com a Amazon
7,a formatação desse ebook atrapalha muito a lei...,Problemas com a Amazon
8,venderam um produto que não tinham para entreg...,Problemas com a Amazon
9,livro pessimo dinheiro jogado fora só mesmo os...,Comentários sobre o livro


In [405]:
test['Mensagem'] = test['Mensagem'].astype('category')
test['Target'] = test['Target'].astype('category')

test['Target'] = test['Target'].cat.rename_categories(['Problemas com a Amazon', 'Comentários sobre o livro', 'Outros'])
test.Target

0      Comentários sobre o livro
1      Comentários sobre o livro
2      Comentários sobre o livro
3      Comentários sobre o livro
4      Comentários sobre o livro
                 ...            
195    Comentários sobre o livro
196    Comentários sobre o livro
197                       Outros
198    Comentários sobre o livro
199    Comentários sobre o livro
Name: Target, Length: 200, dtype: category
Categories (3, object): ['Problemas com a Amazon', 'Comentários sobre o livro', 'Outros']

### Tabela de frequências das palavras

In [406]:
tabela_relativa = train.Target.value_counts(True)

P_amazon = tabela_relativa['Problemas com a Amazon']
P_livro = tabela_relativa['Comentários sobre o livro']
P_outros = tabela_relativa['Outros']
tabela_relativa

Target
Comentários sobre o livro    0.686667
Problemas com a Amazon       0.213333
Outros                       0.100000
Name: proportion, dtype: float64

In [407]:
# Criando uma lista com todas as palavras diferentes
palavras = ''

for i in range(len(train['Mensagem'])):
    palavras += train['Mensagem'][i] + ' '

palavras = palavras.split()
palavras = pd.Series(list(set(palavras)))
n_palavras_diff = len(palavras)

In [408]:
# Criando uma lista com todas as palavras na categoria Amazon

palavras_amazon = ''

amazon = train.loc[train['Target'] == 'Problemas com a Amazon', ['Mensagem']]
amazon = amazon.reset_index(drop=True)

for i in range(len(amazon['Mensagem'])):
    palavras_amazon += amazon['Mensagem'][i] + ' '

palavras_amazon_lista = palavras_amazon.split()
palavras_amazon = pd.Series(palavras_amazon_lista)
P_palavra_dado_amazon = palavras_amazon.value_counts()
n_palavras_amazon = len(palavras_amazon)

In [409]:
# Criando uma lista com todas as palavras na categoria Livros

palavras_livro = ''

livro = train.loc[train['Target'] == 'Comentários sobre o livro', ['Mensagem']]
livro = livro.reset_index(drop=True)

for i in range(len(livro['Mensagem'])):
    palavras_livro += livro['Mensagem'][i] + ' '

palavras_livro_lista = palavras_livro.split()
palavras_livro = pd.Series(palavras_livro_lista)
P_palavra_dado_livro = palavras_livro.value_counts()
n_palavras_livro = len(palavras_livro)

In [410]:
# Criando uma lista com todas as palavras diferentes na categoria Outros

palavras_outros = ''

outros = train.loc[train['Target'] == 'Outros', ['Mensagem']]
outros = outros.reset_index(drop=True)

for i in range(len(outros['Mensagem'])):
    palavras_outros += outros['Mensagem'][i] + ' '

palavras_outros_lista = palavras_outros.split()
palavras_outros = pd.Series(palavras_outros_lista)
P_palavra_dado_outros = palavras_outros.value_counts()
n_palavras_outros = len(palavras_outros)

In [411]:
for i, frase in test['Mensagem'].items():
    frase = frase.split()
    prob_amazon = 1
    prob_livro = 1
    prob_outros = 1
    for palavra in frase:
        if palavra not in palavras_amazon_lista:
            prob_amazon *= (0 + 1)/(n_palavras_amazon + n_palavras_diff)
        else:
            prob_amazon *= (P_palavra_dado_amazon[palavra] + 1)/(n_palavras_amazon + n_palavras_diff)
        
        if palavra not in palavras_livro_lista:
            prob_livro *= (0 + 1)/(n_palavras_livro + n_palavras_diff)
        else:
            prob_livro *= (P_palavra_dado_livro[palavra] + 1)/(n_palavras_livro + n_palavras_diff)
        
        if palavra not in palavras_outros_lista:
            prob_outros *= (0 + 1)/(n_palavras_outros + n_palavras_diff)
        else:
            prob_outros *= (P_palavra_dado_outros[palavra] + 1)/(n_palavras_outros + n_palavras_diff)
    
    if prob_amazon > prob_livro and prob_amazon > prob_outros:
        test.loc[i, 'Predição'] = 'Problemas com a Amazon'
    elif prob_livro > prob_amazon and prob_livro > prob_outros:
        test.loc[i, 'Predição'] = 'Comentários sobre o livro'
    else:
        test.loc[i, 'Predição'] = 'Outros'

In [412]:
pd.crosstab(test['Predição'], test['Target'])

Target,Problemas com a Amazon,Comentários sobre o livro,Outros
Predição,,,
Comentários sobre o livro,2,113,14
Outros,4,14,3
Problemas com a Amazon,33,6,11


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**